In [79]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os


In [80]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/train.csv")
test  = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/test.csv")

print(train.columns)
print(test.columns)

print(train.head())
print(test.head())


Index(['Unnamed: 0', 'file_name', 'label'], dtype='object')
Index(['id'], dtype='object')
   Unnamed: 0                                        file_name  label
0           0  train_data/a6dcb93f596a43249135678dfcfc17ea.jpg      1
1           1  train_data/041be3153810433ab146bc97d5af505c.jpg      0
2           2  train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg      1
3           3  train_data/8542fe161d9147be8e835e50c0de39cd.jpg      0
4           4  train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1
                                                  id
0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3  test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4  test_data_v2/a16495c578b7494683805484ca27cf9f.jpg


In [81]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/train.csv")
test  = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/test.csv")

train['filepath'] = train['file_name'].apply(lambda x: "/content/train_data/" + x)
test['filepath'] = test['id'].apply(
    lambda x: "/content/test_data_v2/" + x
)




train.head(), test.head()


(   Unnamed: 0                                        file_name  label  \
 0           0  train_data/a6dcb93f596a43249135678dfcfc17ea.jpg      1   
 1           1  train_data/041be3153810433ab146bc97d5af505c.jpg      0   
 2           2  train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg      1   
 3           3  train_data/8542fe161d9147be8e835e50c0de39cd.jpg      0   
 4           4  train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1   
 
                                             filepath  
 0  /content/train_data/train_data/a6dcb93f596a432...  
 1  /content/train_data/train_data/041be3153810433...  
 2  /content/train_data/train_data/615df26ce9494e5...  
 3  /content/train_data/train_data/8542fe161d9147b...  
 4  /content/train_data/train_data/5d81fa12bc3b4ce...  ,
                                                   id  \
 0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg   
 1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg   
 2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7

In [82]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train,
    test_size=0.2,
    random_state=42,
    stratify=train['label']
)


In [83]:
from PIL import Image
import os

bad_images = []
root = "/content/train_data/train_data"  # your actual train directory

for fname in os.listdir(root):
    fpath = os.path.join(root, fname)
    try:
        img = Image.open(fpath)
        img.verify()   # check integrity
    except Exception as e:
        bad_images.append(fpath)

len(bad_images), bad_images[:10]


(0, [])

In [84]:
for f in bad_images:
    os.remove(f)

print("Removed", len(bad_images), "bad images.")


Removed 0 bad images.


In [85]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(train_df, val_df, test_df):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        zoom_range=0.1
    )

    test_val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='filepath',
        y_col='label',
        target_size=(160, 160),
        class_mode='raw',
        batch_size=16,
        shuffle=True
    )

    val_generator = test_val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='filepath',
        y_col='label',
        target_size=(160, 160),
        class_mode='raw',
        batch_size=16,
        shuffle=False
    )

    test_generator = test_val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepath',
        y_col=None,
        target_size=(160, 160),
        class_mode=None,
        batch_size=16,
        shuffle=False
    )

    return train_generator, val_generator, test_generator


train_gen, val_gen, test_gen = create_generators(train_df, val_df, test)


Found 63959 validated image filenames.
Found 15990 validated image filenames.
Found 5540 validated image filenames.


/usr/local/lib/python3.12/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

IMG_SIZE = (160, 160)

def create_cnn_batchNormalization_dropout():
    model = keras.Sequential([

        keras.layers.Input(shape=(224, 224, 3)),

        layers.Conv2D(32, (3,3), padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(128, (3,3), padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation="relu"),

        layers.BatchNormalization(),
        layers.Dropout(0.4),

        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model

model_3 = create_cnn_batchNormalization_dropout()
model_3.summary()

history_3 = model_3.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,449 (501.75 KB)

 Trainable params: 127,489 (498.00 KB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 573s 142ms/step - accuracy: 0.8630 - loss: 0.3365 - val_accuracy: 0.7760 - val_loss: 0.5052
Epoch 2/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 555s 139ms/step - accuracy: 0.9095 - loss: 0.2276 - val_accuracy: 0.8756 - val_loss: 0.3194
Epoch 3/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 560s 140ms/step - accuracy: 0.9274 - loss: 0.1886 - val_accuracy: 0.9307 - val_loss: 0.1835
Epoch 4/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 559s 140ms/step - accuracy: 0.9374 - loss: 0.1660 - val_accuracy: 0.8072 - val_loss: 0.5052
Epoch 5/5
 973/3998 ━━━━━━━━━━━━━━━━━━━━ 6:37 131ms/step - accuracy: 0.9500 - loss: 0.1416

In [89]:
print("Final Training Accuracy:", history_3.history["accuracy"][-1])
print("Final Validation Accuracy:", history_3.history["val_accuracy"][-1])


Final Training Accuracy: 0.946590781211853
Final Validation Accuracy: 0.8914321660995483
